In [1]:
import cv2

img = cv2.imread("/content/IMG_5244.JPG")
print(img)


cv2.waitKey(0)
cv2.destroyAllWindows()

None


# New Section

In [4]:
import cv2

from google.colab.patches import cv2_imshow
img = cv2.imread("/content/IMG_5244.JPG")
cv2_imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: ignored

In [5]:
print(f'the size of the image is:{img.shape}')
print(f'the data type of the image is:{img.dtype}')
print(f'the dimension of the image is:{img.ndim}')

AttributeError: ignored

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread("/content/IMG_2419.jpg")

img[50:100,50:100] = 0

cv2_imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()

save image

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread("/content/IMG_2419.jpg")
img[50:100,50:100] = 0
cv2_imshow(img)

cv2.imwrite("result.png",img) #save

cv2.waitKey(0)

cv2.destroyAllWindows()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Color space

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

cap1 = cv2.VideoCapture('/new.mov')

while cap1.isOpened():
  ret, frame = cap1.read()
  if not ret:
    print("can't receive frame (stream end?).")
    break
  cv2_imshow(frame)
  if cv2.waitKey(1) == ord('q'):
    break

cap1.release() #ปล่อย file vdo ออกจาก memory
cv2.destroyAllWindows()

In [ ]:
#capture webcam video

import cv2
from google.colab.patches import cv2_imshow

cap1 = cv2.VideoCapture(0) #0 คือ defeat

while cap1.isOpened():
  ret, frame = cap1.read()
  if not ret:
    print("can't receive frame (stream end?).")
    break
  cv2_imshow(frame)
  if cv2.waitKey(1) == ord('q'):
    break

cv2.waitKey(0)
cap1.release() #ปล่อย file vdo ออกจาก memory
cv2.destroyAllWindows()

Face Detection

In [ ]:
#import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))


webcam image

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  # grayscale img
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  print(gray.shape)
  # get face bounding box coordinates using Haar Cascade
  faces = face_cascade.detectMultiScale(gray)
  # draw face bounding box on image
  for (x,y,w,h) in faces:
      img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
  # save image
  cv2.imwrite(filename, img)

  return filename

In [ ]:
try:
  filename = take_photo('photo.jpg')
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))


webcam video

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "Status:";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '' +
          'When finished, click here or on the video to stop this demo';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # get face region coordinates
    faces = face_cascade.detectMultiScale(gray)
    # get face bounding box for overlay
    for (x,y,w,h) in faces:
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

basic Draw operation

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img1 = cv2.imread('/content/IMG_5245.JPG')
cv2_imshow(img1)
#make a copy of the original image
imageRectangle = img1.copy()
#define the starting and end points of the rectangle
start_point = (300,115)
end_point = (475,225)
#draw ther rectangle
cv2.rectangle(imageRectangle, start_point, end_point, (0, 0, 255), thickness= 3, lineType=cv2.LINE_8)
# display output
cv2_imshow(imageRectangle)
cv2.waitKey(0)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img1 = cv2.imread('/content/IMG_5245.JPG')
#cv2_imshow(img1)

#make a copy of the original image
imageCircle = img1.copy()

circle_center = (415,190)
radius = 100

cv2.circle(imageCircle, circle_center, radius, (0, 0, 255), thickness= 3, lineType=cv2.LINE_8)
# display output
cv2_imshow(imageCircle)
cv2.waitKey(0)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img1 = cv2.imread('/content/IMG_5245.JPG')
#cv2_imshow(img1)

#make a copy of the original image
imageLine = img1.copy()

pointA = (200,80)
pointB = (950,80)
#RGB (Red, Green, Blue)

cv2.line(imageLine, pointA, pointB, (255, 255, 0), thickness = 3, lineType = cv2.LINE_AA)
# display output
cv2_imshow(imageLine)
cv2.waitKey(0)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img1 = cv2.imread('/content/IMG_5245.JPG')
#cv2_imshow(img1)

#make a copy of the original image
imageText = img1.copy()
text = 'Hello openCV'

org = (50,350)

pointA = (200,80)
pointB = (950,80)
#RGB (Red, Green, Blue)

cv2.putText(imageText, text, org, fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 9, color = (250,225,100))
cv2_imshow(imageText)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

img = cv2.imread('/content/IMG_2419.jpg')

#cv2_imshow(img)

imagePoly = img.copy()

penta = np.array([[[40,160],[120,100],[200,160],[160,240],[80,240]]], np.int32)
triangle = np.array([[[240, 130], [380, 230], [190, 280]]], np.int32)

cv2.polylines(imagePoly, [triangle], True, (0,255,0), thickness=3)
cv2.polylines(imagePoly, [penta], True, (255,120,255),3)
cv2_imshow(imagePoly)
cv2.waitKey(0)

blur image

In [ ]:
import cv2

from google.colab.patches import cv2_imshow

img = cv2.imread('/content/IMG_2419.jpg')

smoothing = cv2.blur(img, (5, 5))
cv2_imshow(smoothing)

cv2.waitKey()
cv2.destroyAllWindows()


In [ ]:
smoothing = cv2.GaussianBlur(img, (5, 5), 0)
cv2_imshow(smoothing)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
smoothing = cv2.bilateralFilter(img, 5, 75, 75)
cv2_imshow(smoothing)
#nice
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
smoothing = cv2.medianBlur(img, 5)
cv2_imshow(smoothing)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
Image thresholding

In [ ]:
import cv2

from google.colab.patches import cv2_imshow

img = cv2.imread('/content/IMG_2419.jpg')

ret, thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
cv2_imshow(thresh1)

cv2.waitKey()
cv2.destroyAllWindows()


In [ ]:
ret, thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
cv2_imshow(thresh2)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
ret, thresh3 = cv2.threshold(img,127,255,cv2.THRESH_TRUNC)
cv2_imshow(thresh3)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
ret, thresh4 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
cv2_imshow(thresh4)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
ret, thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)
cv2_imshow(thresh5)

cv2.waitKey()
cv2.destroyAllWindows()

Morphology Operations

In [ ]:
import cv2

from google.colab.patches import cv2_imshow

img = cv2.imread('/content/morph.png')

morph = cv2.dilate(img, (5,5), iterations = 3)
cv2_imshow(morph)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
morph = cv2.erode(img, (5,5), iterations = 3)
cv2_imshow(morph)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
morph = cv2.morphologyEx(img, cv2.MORPH_OPEN, (5,5))
cv2_imshow(morph)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
morph = cv2.morphologyEx(img, cv2.MORPH_CLOSE, (5,5))
cv2_imshow(morph)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
morph = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, (5,5))
cv2_imshow(morph)

cv2.waitKey()
cv2.destroyAllWindows()